In [1]:
import datajoint as dj
schema = dj.Schema('goaltracking')

[2024-11-04 03:23:24,574][INFO]: Connecting root@localhost:3306
[2024-11-04 03:23:24,622][INFO]: Connected root@localhost:3306


In [2]:
import datajoint as dj
schema = dj.Schema('goaltracking')

import pymysql
import os
pymysql.install_as_MySQLdb()

connection_string = "mysql://{user}:{password}@{host}".format(
    user=os.environ['DJ_USER'],
    host=os.environ['DJ_HOST'],
    password=os.environ['DJ_PASS']
)

%load_ext sql
%sql $connection_string

In [3]:
@schema
class Category(dj.Manual):
    definition = """
    category_name : varchar(60)   # Name of category such as Fun, Reading, PRIMARY KEY
    ---
    """


In [4]:
@schema
class Goal(dj.Manual):
    definition = """
    goal_id : int unsigned # PRIMARY KEY
    ---
    goal_name : varchar(60)  
    category_name: varchar(60)
    description: varchar(100)
    start_date:date
    target_completion_date: date
    challenge_level: int # Levels 1,2,3, 4 and 5
    """

In [6]:
@schema
class Achievement(dj.Manual):
    definition = """
    achievement_id: int # PRIMARY KEY
    status:  Enum('Complete', 'Partial', 'Failed')
    ---
    -> Goal
    achivement_date    : date
    """

In [8]:
Goal.insert1(dict(goal_id = "1",
       goal_name = "standup",
       category_name = "Fun",
       description = "This is an attempt at standup comedy. Will practice writing jokes for standup comedy",
       start_date = "2024-01-01",
       target_completion_date = "2024-12-31",
       challenge_level = "1"
    )
)

In [10]:
Goal.insert1(dict(goal_id = "2",
       goal_name = "Meet Sales Quarter",
       category_name = "Work",
       description = "Will Ask every customer if they would like to apply for our Store Credit and Perks ",
       start_date = "2024-01-01",
       target_completion_date = "2024-12-31",
       challenge_level = "2"
    )
) 

In [11]:
Goal.insert1(dict(goal_id = "3",
       goal_name = "Complete October Coursework",
       category_name = "Study",
       description = "Work through all the assignemnts for October and submit ",
       start_date = "2024-10-01",
       target_completion_date = "2024-10-31",
       challenge_level = "3"
    )
) 

In [12]:
Goal.insert1(dict(goal_id = "4",
       goal_name = "Best Sellers",
       category_name = "Reading",
       description = "Read one Best Seller a Month ",
       start_date = "2024-10-01",
       target_completion_date = "2024-10-31",
       challenge_level = "4"
    )
)

In [13]:
Goal.insert1(dict(goal_id = "5",
       goal_name = "Mile Jogging",
       category_name = "Exercise",
       description = "Jog at least one mile once a week ",
       start_date = "2024-10-01",
       target_completion_date = "2024-12-31",
       challenge_level = "5"
    )
) 


In [14]:
Achievement.insert1(dict(achievement_id = "1", status = 'Partial', goal_id = "1",achivement_date = "2024-10-05")) 
Achievement.insert1(dict(achievement_id = "2", status = 'Partial', goal_id = "2",achivement_date = "2024-10-10")) 
Achievement.insert1(dict(achievement_id = "3", status = 'Complete', goal_id = "3",achivement_date = "2024-10-26")) 
Achievement.insert1(dict(achievement_id = "4", status = 'Failed', goal_id = "4",achivement_date = "2024-10-31")) 
Achievement.insert1(dict(achievement_id = "5", status = 'Partial', goal_id = "5",achivement_date = "2024-10-31")) 

In [15]:
Category.insert1(dict(category_name = "Fun"))
Category.insert1(dict(category_name = "Work"))
Category.insert1(dict(category_name = "Study"))
Category.insert1(dict(category_name = "Reading"))
Category.insert1(dict(category_name = "Exercise"))

In [18]:
Category()

"category_name Name of category such as Fun, Reading, PRIMARY KEY"
Exercise
Fun
Reading
Study
Work


In [19]:
Goal()

goal_id PRIMARY KEY,goal_name,category_name,description,start_date,target_completion_date,"challenge_level Levels 1,2,3, 4 and 5"
1,standup,Fun,This is an attempt at standup comedy. Will practice writing jokes for standup comedy,2024-01-01,2024-12-31,1
2,Meet Sales Quarter,Work,Will Ask every customer if they would like to apply for our Store Credit and Perks,2024-01-01,2024-12-31,2
3,Complete October Coursework,Study,Work through all the assignemnts for October and submit,2024-10-01,2024-10-31,3
4,Best Sellers,Reading,Read one Best Seller a Month,2024-10-01,2024-10-31,4
5,Mile Jogging,Exercise,Jog at least one mile once a week,2024-10-01,2024-12-31,5


In [20]:
Achievement()

achievement_id PRIMARY KEY,status,goal_id PRIMARY KEY,achivement_date
1,Partial,1,2024-10-05
2,Partial,2,2024-10-10
3,Complete,3,2024-10-26
4,Failed,4,2024-10-31
5,Partial,5,2024-10-31


In [26]:
# Query to join Goal and Achievement and filter for status 'Complete'
query = (Goal * Achievement & 'status="Complete"').proj('achievement_id', 'description').fetch()

# View the results
print(query)

[(3, 3, 'Complete', 'Work through all the assignemnts for October and submit ')]


In [32]:
(Achievement * Goal).proj('category_name', 'status')

achievement_id PRIMARY KEY,status,goal_id PRIMARY KEY,category_name
1,Partial,1,Fun
2,Partial,2,Work
3,Complete,3,Study
4,Failed,4,Reading
5,Partial,5,Exercise


In [34]:
Achievement.proj().aggr((Achievement * Goal), 'category_name', avgRateOfCompletion='SUM(CASE WHEN status="Complete" THEN 1 ELSE 0 END) / COUNT(status)')

achievement_id PRIMARY KEY,status,category_name,avgRateOfCompletion calculated attribute
1,Partial,Fun,0.0000
2,Partial,Work,0.0000
3,Complete,Study,1.0000
4,Failed,Reading,0.0000
5,Partial,Exercise,0.0000
